# signac - PyData Ann Arbor MeetUp 2018

*The following cell resets all data from previous runs of this notebook.*

In [ ]:
!rm -rf workspace signac.rc project.py *.err.* *.out.* signac_project_document.json view

# Section 1: Introduction

## About
This notebook gives an example of how the ``signac`` framework can be used to manage a data space and automate operations on this data space.
In this example, let's imagine that we're studying the behavior of a projectile launched at a specific velocity and angle to visualize the distance it will travel before it lands.
We use simple Newtonian mechanics to model the motion to determine how long the object travels: 

$
\begin{equation}
    \begin{aligned}
        y(t) &= y(0) + v\sin(\theta) t - \frac{1}{2} g t^2 \\
    \end{aligned}
\end{equation}
$

Setting $y(0)=0$ and solving for $t$ such that $y(t) = 0$, yields: $t_\max= \frac{2v \sin(\theta)}{g}$

## Initial experiments

We express the simple math from above in two Python functions that calculate the maximum time the rocket travels ($t_\max$) and the $xy$-coordinates of its trajectory.

In [ ]:
import numpy as np

def get_t_max(v, theta, g=9.81):
    return 2 * v * np.sin(theta) / g

def compute_xy(t, v, theta, g=9.81):
    return v * np.cos(theta) * t, v * np.sin(theta) * t - (g/2) * t**2

Let's observe the effect of chosing different velocities and launch angles:

In [ ]:
v = 2000 # m/s
theta = 20 * np.pi / 180

t_max = get_t_max(v=v, theta=theta)
print("Time traveled (theta={:2.1f}): {:.2f} min".format(theta * 180/np.pi, t_max / 60))

We can also execute a slightly more "*systematic*" study of the maximum traveled over different launch angles:

In [ ]:
for theta in np.arange(0.0, np.pi/2, 0.2):
    t_max = get_t_max(2000, theta)
    x_max = compute_xy(t_max, 2000, theta)[0]
    print("Distance traveled (theta={:04.1f}): {:3.2f} km".format(theta * 180/np.pi, x_max / 1000))

There is clearly a maximum here somewhere...

It might help to visualize the problem:

In [ ]:
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed
%matplotlib inline

@interact(v=fixed(2000), theta=(0, np.pi/2, 0.1))
def plot(v=2000, theta=0.2):
    t = np.linspace(0, get_t_max(v, theta), 100)
    xy = np.asarray(compute_xy(t, v, theta)) / 1000

    fig, ax = plt.subplots(figsize=(12, 9))
    ax.set_aspect('equal')
    ax.set_title("v={} m/s theta={:.1f}".format(v, theta * 180 / np.pi))
    ax.set_xlabel('x [km]')
    ax.set_ylabel('y [km]')
    ax.set_xlim(0, 1000)
    ax.set_ylim(0, 500)
    ax.plot(* xy, ls=':')
    plt.show()

Essentially the same implementation is in the `render.py` module:

In [ ]:
from render import plot

fig, ax = plot(velocity=2000, theta=80 * np.pi / 180)
plt.show()

We can also create a movie that might help us to understand the problem even better:

In [ ]:
from IPython.display import HTML
from render import animate

anim = animate(velocity=2000, theta=45 * np.pi / 180)
HTML(anim.to_html5_video())

## Initialize a data space

So far so good, but now, let's see how we can manage this data with ``signac``. 

In [ ]:
import signac

# We start by initializing a project
project = signac.init_project("Projectile-Project")

# Obtain a 'job' handle for a specific state point:
job = project.open_job({"theta": 0.4})

# JSON-encodable data can be stored in the *job document*, which works like a persistent dict:
job.doc['t_max'] = get_t_max(v=2000, theta=job.sp['theta'])

# Just like the the *state point*, the document data can also be accessed via *attributes*:
job.doc.x_max = compute_xy(t=job.doc.t_max, v=2000, theta=job.sp.theta)[0]

In [ ]:
print(job.sp)
print(job.doc)

This created the following directory structure on the file-system:

In [ ]:
! find . -not -path '*/[\._]*'

In [ ]:
print(job)
print(job.sp)

## Expand data space

We've shown how this works for one data point.

However, **signac** is designed to interact with large data space, with lots and lots of data points.
This is useful, for example to conduct a parameter study of various launch angles:

In [ ]:
for theta in 0.4, 0.625, 0.85, 1.3:
    job = project.open_job({"theta": theta})
    job.doc['t_max'] = get_t_max(v=2000, theta=job.sp['theta'])
    job.doc.x_max = compute_xy(t=job.doc.t_max, v=2000, theta=job.sp.theta)[0]

## Accessing this data

The data is stored on disc and can be accessed later, for example, by iterating over the entire project.

In [ ]:
x_max = 0
theta_max = None

for job in project:
    if job.doc.x_max > x_max:
        x_max = job.doc.x_max
        theta_max = job.sp.theta

print("The furthest distance traveled was {:3.2f}km with \u03b8={:04.1f}\u00b0.".format(
    x_max/1000, theta_max*180/np.pi))

## Changing the schema

Now imagine that we suddenly discover new fuels for our rocket that allow it to travel much faster than it originally did.
This means that we now have to account for a range of velocities in our data schema.

This is the current *implicit* state point schema:

In [ ]:
print(project.detect_schema())

We can migrate the schema by manipulating the `job.sp`/`job.statepoint` dictionary directly.
First, we move the velocity explicitly into the schema:

In [ ]:
for job in project:
    job.sp.setdefault('velocity', 2000)

Let's add the two additional speeds:

In [ ]:
velocities = [2000, 2500, 3000]
thetas = {job.sp.theta for job in project}

for velocity in velocities:
    for theta in thetas:
        job = project.open_job({'theta': theta, 'velocity': velocity})
        job.init()  # This function is idempotent and won't affect pre-existing jobs!

In [ ]:
print(project.detect_schema())

## Encoding the workflow

Since we're now working with a larger data space, it is a good idea to automate our workflow.
For this we define a `FlowProject` and functions that *operate* on the data space as part of a workflow.

In [ ]:
%%writefile project.py
from flow import FlowProject
import numpy as np


def get_t_max(v, theta, g=9.81):
    return 2 * v * np.sin(theta) / g


def compute_xy(t, v, theta, g=9.81):
    return v * np.cos(theta) * t, v * np.sin(theta) * t - (g/2) * t**2


from flow import FlowProject

class MyProject(FlowProject):
    pass


@MyProject.label
def trajectory_computed(job):
    return job.isfile('trajectory.npz')


@MyProject.operation
@MyProject.post(trajectory_computed)
def compute_trajectory(job):
    t = np.linspace(0, get_t_max(job.sp.velocity, job.sp.theta), 100)
    xy = np.asarray(compute_xy(t, job.sp.velocity, job.sp.theta)).T
    np.savez(job.fn('trajectory.npz'), t=t, xy=xy)
    
    job.doc.t_max = t.max()
    job.doc.x_max = xy[0].max()


if __name__ == '__main__':
    MyProject().main()

In [ ]:
!python3 project.py status -d --pretty --parameters velocity theta

In [ ]:
!python3 project.py -v run -n 3

In [ ]:
!python3 project.py status -d --pretty --parameters velocity theta

In [ ]:
%%writefile project.py
from flow import FlowProject
import numpy as np


def get_t_max(v, theta, g=9.81):
    return 2 * v * np.sin(theta) / g


def compute_xy(t, v, theta, g=9.81):
    return v * np.cos(theta) * t, v * np.sin(theta) * t - (g/2) * t**2


from flow import FlowProject

class MyProject(FlowProject):
    pass


@MyProject.label
def trajectory_computed(job):
    return job.isfile('trajectory.npz')


@MyProject.operation
@MyProject.post(trajectory_computed)
def compute_trajectory(job):
    t = np.linspace(0, get_t_max(job.sp.velocity, job.sp.theta), 100)
    xy = np.asarray(compute_xy(t, job.sp.velocity, job.sp.theta)).T
    np.savez(job.fn('trajectory.npz'), t=t, xy=xy)
    
    job.doc.t_max = t.max()
    job.doc.x_max = xy[0].max()


@MyProject.operation
@MyProject.pre.after(compute_trajectory)
@MyProject.post.isfile('trajectory.png')
def plot_trajectory(job):
    from render import plot
    fig, ax = plot(velocity=job.sp.velocity, theta=job.sp.theta)
    fig.savefig(job.fn('trajectory.png'))


@MyProject.operation
@MyProject.pre.after(compute_trajectory)
@MyProject.post.isfile('trajectory.mp4')
def animate(job):
    from render import animate
    anim = animate(** job.sp)
    with job:
        anim.save('trajectory.mp4')


if __name__ == '__main__':
    MyProject().main()

In [ ]:
!python3 project.py status -d --pretty --parameters velocity theta --stack --all-operations

In [ ]:
!python3 project.py run -o compute_trajectory --progress

In [ ]:
!python3 project.py run --progress --parallel

In [ ]:
!python3 project.py status -d --only-incomplete-operations

## Extract outputs

Now that all this data has been generated it is very easy to access it in the context of ``signac`` and pull the relevant files. 

In [ ]:
import os

html_str= '<table>'

for v, group in project.groupby('velocity'):
    html_str += '<tr>'
    for job in sorted(group, key=lambda job: job.sp.theta):
        html_str += '<td><video width="200" autoplay loop><source src="{}" type="video/mp4"></video></td>'.format(
            os.path.relpath(job.fn('trajectory.mp4')))
    html_str += '</tr>'
html_str += '</table>'

HTML(html_str)

## Viewing the data

Now, this form of data storage may be cleaner in some ways, but now it's completely impossible to inspect the data space manually if you wanted to.
If you wanted to look through the filesystem to see things, you would have to look through each JSON file for the relevant metadata, which really isn't feasible.
To overcome this, we have views.

In [ ]:
project = signac.get_project()
project.create_linked_view()

**Views are dynamic links, so they are immediately updated when the data space changes.
As a result, they can also be easily customized by simply changing the order in which directory structures are constructed when views are created without affecting the data.**

In [ ]:
!find view | head -n 20

In [ ]:
!find -L view | head -10

## Extra

### Submission to scheduler

It is now trivial to submit these jobs to a cluster instead of running them locally.
All we need to do is change our run command to a submit command, and the operation will run on any detected scheduler.

In [ ]:
import os

os.environ['SIMPLE_SCHEDULER'] = "simple-scheduler --data=$HOME/.local/share/simple-scheduler"

In [ ]:
!find . -name trajectory.mp4 | head -4 | xargs rm

In [ ]:
!python3 project.py status -d --pretty --parameters velocity theta --stack --all-operations

In [ ]:
!python3 project.py submit